
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# 画像の畳み込み

:label: `sec_conv_layer`

畳み込み層が理論的にどのように機能するかを理解したので、実際にどのように機能するかを確認する準備が整いました。画像データの構造を探索するための効率的なアーキテクチャとしての畳み込みニューラル ネットワークの動機に基づいて、私たちは実行例として画像にこだわります。


In [1]:
import torch
from torch import nn
from d2l import torch as d2l


## 相互相関演算

厳密に言えば、畳み込み層が表現する演算は相互相関としてより正確に記述されるため、畳み込み層は誤った呼び名であることを思い出してください。 :numref: `sec_why-conv`の畳み込み層の説明に基づくと、このような層では、入力テンソルとカーネル テンソルが結合され、(**相互相関演算) を通じて出力テンソルが生成されます。**

ここではチャネルを無視して、これが 2 次元データと非表示表現でどのように機能するかを見てみましょう。 :numref: `fig_correlation`では、入力は高さ 3、幅 3 の 2 次元テンソルです。テンソルの形状を $3 \times 3$ または ($3$, $3$) としてマークします。カーネルの高さと幅は両方とも 2 です。*カーネル ウィンドウ*(または*畳み込みウィンドウ*) の形状は、カーネルの高さと幅 (ここでは $2 \times 2$) によって与えられます。 

![](http://d2l.ai/_images/correlation.svg) :label: `fig_correlation`

 2 次元相互相関演算では、入力テンソルの左上隅に配置された畳み込みウィンドウから開始し、それを入力テンソル上で左から右、上から下の両方にスライドさせます。畳み込みウィンドウが特定の位置にスライドすると、そのウィンドウに含まれる入力サブテンソルとカーネル テンソルが要素ごとに乗算され、結果のテンソルが合計されて 1 つのスカラー値が生成されます。この結果により、対応する位置での出力テンソルの値が得られます。ここで、出力テンソルの高さは 2、幅は 2 で、4 つの要素は 2 次元相互相関演算から導出されます。

 $$ 0\times0+1\times1+3\times2+4\times3=19,\ 1\times0+2\times1+4\times2+5\times3=25,\ 3\times0+4\times1+6\回 2+7\回 3=37、\ 4\回 0+5\回 1+7\回 2+8\回 3=43。 $$

各軸に沿って、出力サイズが入力サイズよりわずかに小さいことに注意してください。カーネルの幅と高さは 1 より大きいため、カーネルが画像内に完全に収まる位置の相互相関のみを適切に計算できます。出力サイズは、入力サイズ $n_h \times n_w$ から次のサイズを引いたもので求められます。畳み込みカーネル $k_h \times k_w$

 $$(n_h-k_h+1) \times (n_w-k_w+1).$$

画像全体でコンボリューション カーネルを「シフト」するのに十分なスペースが必要なため、これが当てはまります。後で、カーネルをシフトするのに十分なスペースを確保するために、画像の境界の周りにゼロを埋め込み、サイズを変更しないようにする方法を説明します。次に、このプロセスを`corr2d`関数に実装します。この関数は、入力テンソル`X`とカーネル テンソル`K`を受け入れ、出力テンソル`Y`を返します。


In [2]:
def corr2d(X, K):  #@save
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y


:numref: `fig_correlation`から入力テンソル`X`とカーネル テンソル`K`を構築して、2 次元相互相関演算の [**上記の実装の出力を検証**] することができます。


In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])


## 畳み込み層

畳み込み層は入力とカーネルを相互相関させ、スカラー バイアスを追加して出力を生成します。畳み込み層の 2 つのパラメーターは、カーネルとスカラー バイアスです。畳み込み層に基づいてモデルをトレーニングする場合、通常、完全接続層の場合と同様に、カーネルをランダムに初期化します。

これで、上で定義した`corr2d`関数に基づいて [ **2 次元畳み込み層を実装する**] 準備が整いました。 `__init__`コンストラクター メソッドでは、 `weight`と`bias` 2 つのモデル パラメーターとして宣言します。順伝播メソッドは、 `corr2d`関数を呼び出してバイアスを追加します。


In [4]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias


$h \times w$ 畳み込みまたは $h \times w$ 畳み込みカーネルでは、畳み込みカーネルの高さと幅はそれぞれ $h$ と $w$ です。 $h \times w$ 畳み込みカーネルを備えた畳み込み層を単に $h \times w$ 畳み込み層とも呼びます。

## 画像内のオブジェクトのエッジ検出

ピクセル変化の位置を見つけて [**畳み込み層の簡単な応用: 画像内のオブジェクトのエッジを検出する**] を解析してみましょう。まず、$6\times 8$ ピクセルの「画像」を構築します。中央の 4 列は黒 (0)、残りは白 (1) です。


In [5]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])


次に、高さ 1、幅 2 のカーネル`K`を構築します。入力に対して相互相関演算を実行すると、水平方向に隣接する要素が同じ場合、出力は 0 になります。それ以外の場合、出力は非になります。 -ゼロ。このカーネルは有限差分演算子の特殊なケースであることに注意してください。位置 $(i,j)$ で $x_{i,j} - x_{(i+1),j}$ を計算します。つまり、水平方向に隣接するピクセルの値の差を計算します。これは、水平方向の一次導関数の離散近似です。結局のところ、関数 $f(i,j)$ の導関数 $-\partial_i f(i,j) = \lim_{\epsilon \to 0} \frac{f(i,j) - f(i+\イプシロン,j)}{\イプシロン}$。これが実際にどのように機能するかを見てみましょう。


In [6]:
K = torch.tensor([[1.0, -1.0]])


引数`X` (入力) と`K` (カーネル) を使用して相互相関演算を実行する準備ができました。ご覧のとおり、[**白から黒へのエッジには 1 が検出され、黒から白へのエッジには -1 が検出されます。** ] 他のすべての出力は値 0 をとります。


In [7]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


これで、転置された画像にカーネルを適用できるようになりました。予想通り、消えてしまいます。 [**カーネル`K`垂直エッジのみを検出します。** 】


In [8]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


## カーネルを学ぶ

有限差分`[1, -1]`によるエッジ検出器の設計は、これがまさに探しているものであることがわかっている場合には適切です。ただし、より大きなカーネルを見て、連続する畳み込み層を検討すると、各フィルターが何を行うべきかを手動で正確に指定するのは不可能になる可能性があります。

ここで、入力と出力のペアのみを見て [ **`X`から`Y`を生成したカーネルを学習する**] ことができるかどうかを見てみましょう。まず畳み込み層を構築し、そのカーネルをランダム テンソルとして初期化します。次に、各反復で二乗誤差を使用して`Y`畳み込み層の出力と比較します。その後、カーネルを更新するための勾配を計算できます。簡単にするために、以下では 2 次元畳み込み層の組み込みクラスを使用し、バイアスを無視します。


In [9]:
# Construct a two-dimensional convolutional layer with 1 output channel and a
# kernel of shape (1, 2). For the sake of simplicity, we ignore the bias here
conv2d = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False)

# The two-dimensional convolutional layer uses four-dimensional input and
# output in the format of (example, channel, height, width), where the batch
# size (number of examples in the batch) and the number of channels are both 1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # Learning rate

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # Update the kernel
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

epoch 2, loss 6.453
epoch 4, loss 1.491
epoch 6, loss 0.418
epoch 8, loss 0.139
epoch 10, loss 0.051



10 回の反復後に誤差が小さな値に下がっていることに注意してください。ここで、[**学習したカーネル テンソルを見てみましょう。** 】


In [10]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0112, -0.9661]])


実際、学習されたカーネル テンソルは、以前に定義したカーネル テンソル`K`に非常に近いです。

## 相互相関と畳み込み

:numref: `sec_why-conv`での相互相関演算と畳み込み演算の間の対応関係の観察を思い出してください。ここで、引き続き 2 次元畳み込み層について考えてみましょう。このような層が相互相関ではなく、:eqref: `eq_2d-conv-discrete`で定義されている厳密な畳み込み演算を実行するとどうなるでしょうか?厳密な*畳み込み*演算の出力を取得するには、2 次元のカーネル テンソルを水平方向と垂直方向の両方に反転し、入力テンソルを使用して*相互相関*演算を実行するだけです。

カーネルは深層学習のデータから学習されるため、畳み込み層が厳密な畳み込み演算または相互相関演算を実行しても、その出力は影響を受けないことに注意してください。

これを説明するために、畳み込み層が*相互相関を*実行し、 :numref: `fig_correlation`でカーネルを学習すると仮定します。これは、ここでは行列 $\mathbf{K}$ として示されています。他の条件が変わらないと仮定すると、代わりにこの層が厳密な*畳み込みを*実行すると、学習されたカーネル $\mathbf{K}&#39;$ は、$\mathbf{K}&#39;$ が両方とも反転された後の $\mathbf{K}$ と同じになります。水平方向と垂直方向。つまり、畳み込み層が :numref: `fig_correlation`と $\mathbf{K}&#39;$ の入力に対して厳密な*畳み込みを*実行すると、同じ出力が :numref: `fig_correlation` (入力と $\mathbf{ K}$)が得られます。

深層学習の文献の標準用語に従って、厳密に言えば多少異なりますが、相互相関演算を引き続き畳み込みと呼びます。さらに、レイヤー表現またはコンボリューション カーネルを表すテンソルのエントリ (またはコンポーネント) を指すために、要素*という用語*を使用します。

## 特徴マップと受容野

:numref: `subsec_why-conv-channels`で説明されているように、 :numref: `fig_correlation`の畳み込み層の出力は、空間次元 (幅と高さなど) で学習された表現 (特徴) と見なすことができるため、*特徴マップと*呼ばれることもあります。 ) を後続のレイヤーに送ります。 CNN では、ある層の任意の要素 $x$ について、その*受容場は*、順伝播中に $x$ の計算に影響を与える可能性のある (前のすべての層からの) すべての要素を指します。受容野は入力の実際のサイズよりも大きい場合があることに注意してください。

引き続き :numref: `fig_correlation`を使用して、受容野について説明しましょう。 $2 \times 2$ 畳み込みカーネルの場合、陰影付き出力要素 (値 $19$) の受容野は、入力の陰影付き部分の 4 つの要素です。ここで、 $2 \times 2$ の出力を $\mathbf{Y}$ として表し、 $\mathbf{Y}$ を入力として受け取り、単一の要素を出力する追加の $2 \times 2$ 畳み込み層を備えたより深い CNN を考えてみましょう。 $z$。この場合、$\mathbf{Y}$ 上の $z$ の受容野には $\mathbf{Y}$ の 4 つの要素がすべて含まれ、入力上の受容野には 9 つの入力要素すべてが含まれます。したがって、より広い領域にわたって入力特徴を検出するために、特徴マップ内のいずれかの要素がより大きな受容野を必要とする場合、より深いネットワークを構築できます。

受容野の名前は神経生理学に由来しています。さまざまな動物とさまざまな刺激に関する一連の実験:cite: `Hubel.Wiesel.1959,Hubel.Wiesel.1962,Hubel.Wiesel.1968`で、ヒューベルとヴィーゼルは、前記刺激に対するいわゆる視覚野の反応を調査しました。 。概して、下位レベルはエッジと関連する形状に反応することがわかりました。その後、:citet: `Field.1987`では、畳み込みカーネルとしか呼ぶことができないものを使用して、自然画像に対するこの効果を説明しました。顕著な類似点を示すために、:numref: `field_visual`の主要な数値を再掲します。 

![](../img/field-visual.png) :label: `field_visual`

結局のところ、この関係は、たとえば :citet: `Kuzovkin.Vicente.Petton.ea.2018`で実証されているように、画像分類タスクで訓練されたネットワークのより深い層によって計算された特徴にも当てはまります。畳み込みは、生物学とコードの両方において、コンピューター ビジョンにとって非常に強力なツールであることが証明されていると言えば十分でしょう。したがって、彼らがディープラーニングにおける最近の成功を予告したことは、（今にして思えば）驚くべきことではありません。

## まとめ

畳み込み層に必要な中心的な計算は相互相関演算です。単純なネストされた for ループだけでその値を計算できることが分かりました。複数の入力チャネルと複数の出力チャネルがある場合、チャネル間で行列間演算を実行することになります。ご覧のとおり、計算は簡単で、最も重要なことに、非常に*局所的です*。これにより、ハードウェアの大幅な最適化が可能になり、コンピューター ビジョンにおける最近の成果の多くは、それによってのみ可能になります。結局のところ、これは、チップ設計者が畳み込みの最適化に関して、メモリではなく高速計算に投資できることを意味します。これは他のアプリケーションにとって最適な設計にはつながらないかもしれませんが、ユビキタスで手頃な価格のコンピューター ビジョンへの扉を開きます。

畳み込み自体に関しては、エッジや線の検出、画像のぼかし、鮮明化など、さまざまな目的に使用できます。最も重要なことは、統計学者 (またはエンジニア) が適切なフィルターを発明する必要がないということです。代わりに、単純にデータからそれらを*学ぶこと*ができます。これにより、特徴エンジニアリングのヒューリスティックが証拠に基づいた統計に置き換えられます。最後に、非常に嬉しいことに、これらのフィルターは深いネットワークを構築するのに有利なだけでなく、脳内の受容野や特徴マップにも対応します。これにより、私たちは正しい道を進んでいるという自信が得られます。

## 演習
1. 斜めのエッジを持つイメージ`X`を構築します。<ol><li>これにこのセクションのカーネル`K`を適用するとどうなるでしょうか?
1.  `X`を転置するとどうなるでしょうか?
1.  `K`を転置するとどうなりますか?
1. 方向ベクトル $\mathbf{v} = (v_1, v_2)$ を指定すると、$\mathbf{v}$ に直交するエッジ、つまり $(v_2, -v_1) 方向のエッジを検出するエッジ検出カーネルを導出します。 $。
1. 二次導関数の有限差分演算子を導出します。それに関連する畳み込みカーネルの最小サイズはどれくらいですか?画像内のどの構造がそれに最も強く反応しますか?
1. ブラー カーネルをどのように設計しますか?なぜそのようなカーネルを使用したいのでしょうか?
1. 次数 $d$ の導関数を取得するためのカーネルの最小サイズはいくらですか?



[ディスカッション](https://discuss.d2l.ai/t/66)
